In [1]:
# use dir() to see attributes
# use help() to see an object's methods

In [2]:
import operator
import os

import pickle
import pandas as pd
import pytz

from qstrader.data.daily_bar_csv import CSVDailyBarDataSource
from qstrader.alpha_model.alpha_model import AlphaModel
from qstrader.alpha_model.fixed_signals import FixedSignalsAlphaModel
from qstrader.signals.buffer import AssetPriceBuffers
from qstrader.asset.equity import Equity
from qstrader.asset.universe.static import StaticUniverse
from qstrader.signals.signal import Signal 
from qstrader.signals.signals_collection import SignalsCollection
from qstrader.signals.momentum import MomentumSignal
from qstrader.data.backtest_data_handler import BacktestDataHandler
from qstrader.statistics.tearsheet import TearsheetStatistics
from qstrader.trading.backtest import BacktestTradingSession

In [3]:
#strategy symbols
strategy_symbols = ['SPY']
strategy_assets = ['EQ:%s' % symbol for symbol in strategy_symbols]
strategy_universe = StaticUniverse(strategy_assets)
    
# data source to load only those provided symbols
csv_dir = os.environ.get('QSTRADER_CSV_DATA_DIR', '.')
data_source = CSVDailyBarDataSource(csv_dir, Equity, csv_symbols=strategy_symbols)
data_handler = BacktestDataHandler(strategy_universe, data_sources=[data_source])

Loading CSV files into DataFrames...
Loading CSV file for symbol 'EQ:SPY'...
Adjusting pricing in CSV files...
Adjusting CSV file for symbol 'EQ:SPY'...


In [4]:
# I'm unsure if I should call this signals or signal. In the momentum example
# he's using a child class MomentumSignals. I don't need a lookback window,
# so I think I just need to use the parent class

class HiddenMarkovAlphaModel(AlphaModel):
    #keep model static for now, but a look_back parameter could be added to modify the lookback window
    def __init__(self, hmm_model, signals, lookback, universe, data_handler):
        self.hmm_model = hmm_model
        self.signals = signals
        self.lookback = lookback
        self.universe = universe
        self.data_handler = data_handler
    #methods
    # determining the regime is going to have to take time, i just think time
    # as parameters. 
    # time needs to be a parameter so it can look up the price on that day. 
    def determine_regime(
        self, dt
    ):
#             dt : `pd.Timestamp`
#             The datetime for which the prediction
#             should be calculated.
# First we need to select the assets
        assets = self.signals['momentum'].assets
        # then we need to grab the adjust closing returns
        returns = {
            asset: self.signals['momentum'](
                assets, self.lookback
            ) for asset in assets
        }
        #fit the model to the returns
        hidden_state = self.hmm_model.predict(returns)
        return hidden_state
    
    #generate the investment or sell signal. In this case return weight 1
    #if regime is favorable, or weight 0 if the regime isnt favorable
    def _generate_signals(
        self, dt, weights
    ):
        assets = self.signals.assets
        if hidden_state == 0:
            weights[asset] = 1
        else:
            weights[asset] = 0.0
        return weights

#threw this in there because it errored. 
def __call__(
        self, dt
    ):
#         """
#         Calculates the signal weights for the top N
#         momentum alpha model, assuming that there is
#         sufficient data to begin calculating momentum
#         on the desired assets.

#         Parameters
#         ----------
#         dt : `pd.Timestamp`
#             The datetime for which the signal weights
#             should be calculated.

#         Returns
#         -------
#         `dict{str: float}`
#             The newly created signal weights dictionary.
#         """
        assets = self.universe.get_assets(dt)
        return weights


In [5]:
#back_test parameters
if __name__ == "__main__":
    # Duration of the backtest
    start_dt = pd.Timestamp('2006-01-02 14:30:00', tz=pytz.UTC)
    end_dt = pd.Timestamp('2022-04-30 23:59:00', tz=pytz.UTC)
    
# load pickle
infile = open('hmm_model_spy.pkl', 'rb')
hmm_model = pickle.load(infile)

#Generate the signals used in the HMM model
# NOTE: lookback must be passed as a list. lookback of [1] + momentum signal just calculates the returns from the previous day
#which is what HMM predicts off of.  
lookback = [2]
hmm_return_signal = MomentumSignal(start_dt, strategy_universe, lookbacks=lookback)
signals = SignalsCollection({'momentum': hmm_return_signal}, data_handler)

#hidden markov strategy alpha model 
strategy_alpha_model = HiddenMarkovAlphaModel(
    hmm_model, signals, lookback, strategy_universe, data_handler
)

#try changing to long/short later
strategy_backtest = BacktestTradingSession(
    start_dt,
    end_dt,
    strategy_universe,
    strategy_alpha_model,
    signals = signals,
    rebalance='end_of_month',
    long_only=True,
    cash_buffer_percentage=0.01,
    data_handler=data_handler
)
strategy_backtest.run()


Initialising simulated broker "Backtest Simulated Broker Account"...
(2006-01-02 14:30:00+00:00) - portfolio creation: Portfolio "000001" created at broker "Backtest Simulated Broker Account"
(2006-01-02 14:30:00+00:00) - subscription: 1000000.00 subscribed to portfolio "000001"
Beginning backtest simulation...
(2006-01-02 14:30:00+00:00) - market_open
(2006-01-02 21:00:00+00:00) - market_close


C:\Users\bng19\anaconda3_64bit\envs\QuantitativeTrading\lib\site-packages\qstrader\data\daily_bar_csv.py:219: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  bid = bid_ask_df.iloc[bid_ask_df.index.get_loc(dt, method='pad')]['Bid']


(2006-01-03 14:30:00+00:00) - market_open
(2006-01-03 21:00:00+00:00) - market_close
(2006-01-04 14:30:00+00:00) - market_open
(2006-01-04 21:00:00+00:00) - market_close
(2006-01-05 14:30:00+00:00) - market_open
(2006-01-05 21:00:00+00:00) - market_close
(2006-01-06 14:30:00+00:00) - market_open
(2006-01-06 21:00:00+00:00) - market_close
(2006-01-09 14:30:00+00:00) - market_open
(2006-01-09 21:00:00+00:00) - market_close
(2006-01-10 14:30:00+00:00) - market_open
(2006-01-10 21:00:00+00:00) - market_close
(2006-01-11 14:30:00+00:00) - market_open
(2006-01-11 21:00:00+00:00) - market_close
(2006-01-12 14:30:00+00:00) - market_open
(2006-01-12 21:00:00+00:00) - market_close
(2006-01-13 14:30:00+00:00) - market_open
(2006-01-13 21:00:00+00:00) - market_close
(2006-01-16 14:30:00+00:00) - market_open
(2006-01-16 21:00:00+00:00) - market_close
(2006-01-17 14:30:00+00:00) - market_open
(2006-01-17 21:00:00+00:00) - market_close
(2006-01-18 14:30:00+00:00) - market_open
(2006-01-18 21:00:00+00

NotImplementedError: Should implement __call__()

In [ ]:
  # Construct benchmark assets (buy & hold SPY)
    benchmark_symbols = ['SPY']
    benchmark_assets = ['EQ:SPY']
    benchmark_universe = StaticUniverse(benchmark_assets)
    benchmark_data_source = CSVDailyBarDataSource(csv_dir, Equity, csv_symbols=benchmark_symbols)
    benchmark_data_handler = BacktestDataHandler(benchmark_universe, data_sources=[benchmark_data_source])

    # Construct a benchmark Alpha Model that provides
    # 100% static allocation to the SPY ETF, with no rebalance
    benchmark_alpha_model = FixedSignalsAlphaModel({'EQ:SPY': 1.0})
    benchmark_backtest = BacktestTradingSession(
        burn_in_dt,
        end_dt,
        benchmark_universe,
        benchmark_alpha_model,
        rebalance='buy_and_hold',
        long_only=True,
        cash_buffer_percentage=0.01,
        data_handler=benchmark_data_handler
    )
    benchmark_backtest.run()


In [ ]:
#tearsheet end results
    tearsheet = TearsheetStatistics(
        strategy_equity=strategy_backtest.get_equity_curve(),
        benchmark_equity=benchmark_backtest.get_equity_curve(),
        title='US Sector Momentum - Top 3 Sectors'
    )
    tearsheet.plot_results()